In [2]:
#import all packages required
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup

print('library imported successfully')

library imported successfully


In [3]:
#Actually I used Spyder for the coding writing....
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
output = requests.get(url)
data_soup = BeautifulSoup(output.content, 'html.parser')
table = data_soup.find('table')
fields = table.find_all('td')

a = len(fields)
print(a)

postcode = []
borough = []
nebod = []

for i in range(0, a, 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    nebod.append(fields[i+2].text.strip())
        
loc_data = pd.DataFrame(data=[postcode, borough, nebod]).T
loc_data.columns = ['Postcode', 'Borough', 'Neighbourhood']
loc_data.head()

864


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
loc_data['Borough'].replace('Not assigned', np.nan, inplace=True)
loc_data.dropna(subset=['Borough'], inplace=True)

loc_data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
loc_data2 = loc_data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
loc_data2['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)
loc_data2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
geo.columns = ['Postcode', 'Latitude', 'Longitude']
data_clean = pd.merge(loc_data2, geo, how='inner', on=['Postcode'])
data_gtg = data_clean[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()
print(len(data_gtg))
data_gtg.head()

103


,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [9]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="lol")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are ' + str(latitude) + ' and ' + str(longitude))

The coordinates of Toronto are 43.653963 and -79.387207


In [14]:
# create map of Toronto, Canada using latitude and longitude values
map_trt = folium.Map(location=[latitude, longitude], zoom_start= 9.5 )

# add markers to map
for lat, long, borough, neighborhood in zip(data_gtg['Latitude'], data_gtg['Longitude'], data_gtg['Borough'], data_gtg['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_trt)  
    
map_trt

In [22]:
#Start Exploring Toronto downtown area.


CLIENT_ID = '4XBXSKO2M0MAPRWO3LTW55IK0RHJ20XINY4BYNSAMSZG3STF' 
CLIENT_SECRET = 'NSMD1BCKJ2V3AF2PMUPOLJZMOXZ3VN0HCZNQEVAJLG2WYHOS'
VERSION = '20180605' # Foursquare API version

data_downtown = data_gtg[data_gtg['Borough'].str.contains('Downtown')]
print(len(data_downtown))
data_downtown.head()
data_downtown = data_downtown.reset_index(drop=True)
data_downtown.head()

18


,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Downtown Toronto,Rosedale,M4W,43.679563,-79.377529
1,Downtown Toronto,"Cabbagetown, St. James Town",M4X,43.667967,-79.367675
2,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
3,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
4,Downtown Toronto,"Ryerson, Garden District",M5B,43.657162,-79.378937


In [23]:
#Let us go to downtown

# create map of Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, long, borough, neighborhood in zip(data_downtown['Latitude'], data_downtown['Longitude'], data_downtown['Borough'], data_downtown['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [25]:
#4 is my lucky number... so let us go the 4th place
limit = 50
rad = 500

name = data_downtown.loc[3, 'Neighbourhood']
dt_latitude = data_downtown.loc[3, 'Latitude']
dt_longitude = data_downtown.loc[3, 'Longitude']


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    dt_latitude, 
    dt_longitude, 
    rad, 
    limit)
print(name)
url

Harbourfront, Regent Park


'https://api.foursquare.com/v2/venues/explore?&client_id=4XBXSKO2M0MAPRWO3LTW55IK0RHJ20XINY4BYNSAMSZG3STF&client_secret=NSMD1BCKJ2V3AF2PMUPOLJZMOXZ3VN0HCZNQEVAJLG2WYHOS&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=50'

In [46]:
output2 = requests.get(url).json()
output2

{'meta': {'code': 200, 'requestId': '5ca9aadbf594df23ff14e587'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [47]:
place_to_see = output2['response']['groups'][0]['items']
place_to_see = json_normalize(place_to_see)
place_to_see = place_to_see.loc[:, ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]
place_to_see

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",43.653447,-79.362017
1,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",43.653191,-79.357947
3,Morning Glory Cafe,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",43.653947,-79.361149
4,Body Blitz Spa East,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",43.654735,-79.359874
5,Impact Kitchen,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",43.656369,-79.356980
6,Dominion Pub and Kitchen,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",43.656919,-79.358967
7,Figs Breakfast & Lunch,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",43.655675,-79.364503
8,Corktown Common,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.655618,-79.356211
9,The Distillery Historic District,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",43.650244,-79.359323


In [48]:
#clean the data
categories_name = place_to_see['venue.categories']
categories_name[0]

[{'id': '4bf58dd8d48988d16a941735',
  'name': 'Bakery',
  'pluralName': 'Bakeries',
  'shortName': 'Bakery',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
   'suffix': '.png'},
  'primary': True}]

In [49]:
def cat_name(cat_list):
    return cat_list[0]['name']

In [50]:
place_to_see['venue.categories'] = place_to_see['venue.categories'].apply(cat_name)
place_to_see

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
7,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
8,Corktown Common,Park,43.655618,-79.356211
9,The Distillery Historic District,Historic Site,43.650244,-79.359323
